<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

<h2 style="text-align: center;"><b>Домашнее задание: применение Dropout (на примере задачи регрессии)</b></h2>

---

Напомним как выглядит полносвязная нейросеть:

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "http://juice-health.ru/images/technology/32.png")

## Проблема

Переобучение (overfitting) — одна из проблем глубоких нейронных сетей, состоящая в следующем: модель хорошо объясняет только примеры из обучающей выборки, адаптируясь к обучающим примерам, вместо того чтобы учиться классифицировать примеры, не участвовавшие в обучении (теряя способность к обобщению).

In [2]:
Image(url= "https://upload.wikimedia.org/wikipedia/commons/6/68/Overfitted_Data.png")

## Идея

Главная идея Dropout — вместо обучения одной нейронной сети обучить ансамбль нескольких нейронных сетей, а затем усреднить полученные результаты.

## Реализация

Сети для обучения получаются с помощью исключения из сети (dropping out) нейронов с вероятностью $p$(в семинаре $keep\_rate$), таким образом, вероятность того, что нейрон останется в сети, составляет $q = 1 - p$. “Исключение” нейрона означает, что при любых входных данных или параметрах он возвращает 0.

 Слева — нейронная сеть до того, как к ней применили Dropout, справа — та же сеть после Dropout.

In [3]:
Image(url= "https://hsto.org/web/dd8/171/16f/dd817116fc2348e78272577153e31d2d.jpeg")

## Результат

Исключенные нейроны не вносят свой вклад в процесс обучения ни на одном из этапов алгоритма обратного распространения ошибки (backpropagation), поэтому исключение хотя бы одного из нейронов равносильно обучению новой нейронной сети.
В двух словах, Dropout хорошо работает на практике, потому что предотвращает взаимоадаптацию нейронов на этапе обучения.

-----------------------------------------------------------------------------

Посмотрим как хорошо на практике работает Dropout. Будем решать задачу **регресии**. Сгенерируем две выборки $test\_y$, $y$ из одного распределения и будем обучаться на $y$, а смотреть ошибку на $test\_y$. Будем использовать два способа обучения: c использованием Dropout и без него. 

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
%config InlineBackend.figure_format = 'svg' 
torch.manual_seed(2)   
np.random.seed(2)

Мы показываем вам, как были сгенерированы $y$ и $test\_y$, но, чтобы наблюдать правильность выполнения задания, мы зафиксировали какие-то рандомные выборки из этого распределения.

In [ ]:
# Кол-во точек 
N_SAMPLES = 20

# Строим выборку для обучения 
x = torch.linspace(-1, 1, N_SAMPLES).view(N_SAMPLES, -1)
#y = x + 0.3 * torch.normal(torch.zeros(N_SAMPLES, 1), torch.ones(N_SAMPLES, 1))
y = torch.tensor([[-1.3122],[-0.6198],[-1.1807],[-1.0171],[-0.5700],[-0.4886],[-0.0489],[ 0.0027],[-0.4012],[ 0.1495]
                  ,[-0.2844],[ 0.1303],[ 0.3053],[ 0.7042],[ 0.5683],[ 1.1048],[ 0.4623],[ 0.4167],[ 0.8422],[ 1.2097]])

# Строим тестовую выборку
test_x = torch.linspace(-1, 1, N_SAMPLES).view(N_SAMPLES, -1)
#test_y = test_x + 0.3 * torch.normal(torch.zeros(N_SAMPLES, 1), torch.ones(N_SAMPLES, 1))
test_y = torch.tensor([[-1.3883],[-0.9020],[-0.8601],[-0.8968],[-0.2950],[-1.0038],[-0.5611],[ 0.4919],[-0.4285],[-0.0572],
                       [ 0.4683],[ 0.9315],[ 0.1591],[ 0.3946],[ 0.1438],[ 0.7278],[ 1.1072],[ 0.6730],[ 1.0141],[ 0.6687]])

Отобразим наши данные:

In [ ]:
plt.scatter('''your code here''', c='magenta', s=20, alpha=0.6, label='train')
plt.scatter('''your code here''', c='cyan', s=20, alpha=0.6, label='test')
plt.legend(loc='upper left')
plt.ylim((-2.5, 2.5))
plt.show()

 <b>Важно! На вход нам подается один нейрон, так как точка $x$ имеет один признак(координату), всего объектов 20. Для каждого объкта независимо от других прогнозируется его $y$, но линейное преобразование на определенном слое для каждого объекта одинакова, а веса этого слоя определяются из минимизации суммарной ошибки на $y$</b>
 
 Создадим нейронную сеть с 3 слоями и в качестве нелинейности используем ReLU:

In [ ]:
# Размер и входа и выхода линейного скрытого слоя
N_HIDDEN = 300
net_overfitting = torch.nn.Sequential(
    torch.nn.Linear(1, N_HIDDEN),
    '''your code here'''
    torch.nn.Linear(N_HIDDEN, 1),
)
print(net_overfitting)  # архитектура сети

Добавим Dropout после первого и второго слова с keep_rate=0.5, то есть удаляем нейрон с вероятностью 0.5:

In [ ]:
net_dropped = torch.nn.Sequential(
    torch.nn.Linear(1, N_HIDDEN),
    '''your code here'''
    torch.nn.Linear(N_HIDDEN, 1),
)
print(net_dropped) # архитектура сети

Зададим градиентный спуск каждой нейронной сети:

In [ ]:
optimizer_ofit = torch.optim.Adam('''your code here''', lr=0.01)
optimizer_drop = torch.optim.Adam('''your code here''', lr=0.01)

Зададим квадратичную функцию потерь:

In [ ]:
loss_func = torch.nn.'''your code here'''

In [ ]:
# Ошибки на y и test_y
error = []
error_test = []
for t in range(500):
    # Пропустим x по нейронной сети
    pred_ofit = '''your code here'''
    pred_drop = '''your code here'''
    
    loss_ofit = loss_func(pred_ofit, y)
    loss_drop = loss_func(pred_drop, y)

    # Cделаем шаг градиентного спуска
    optimizer_ofit.zero_grad()
    optimizer_drop.zero_grad()
    '''your code here'''

    if t % 20 == 0:
        # перейдем к режиму eval, чтобы учесть поправку dropout
        net_overfitting.eval()
        net_dropped.eval()

        # plotting
        clear_output(wait=True)
        sleep(0.05)
        
         # Пропустим test_x по нейронной сети
        '''your code here'''
        
        #Отобразим наши данные и предсказания различных нейронных сетей на тестовых данных:
        plt.scatter('''your code here''', c='magenta', s=20, alpha=0.6, label='train')
        plt.scatter('''your code here''', c='cyan', s=20, alpha=0.6, label='test')
        plt.plot(test_x.data.numpy(), test_pred_ofit.data.numpy(), 'r-', lw=3, label='overfitting')
        plt.plot(test_x.data.numpy(), test_pred_drop.data.numpy(), 'b--', lw=3, label='dropout(50%)')
        
        # Ошибка на y и на на test_y
        error.append((loss_func(test_pred_ofit, y).data.numpy(), loss_func(test_pred_drop, y).data.numpy()))
        error_test.append('''your code here''')
        
        plt.text(-0.5, -1.8, 'overfitting loss=%.4f' % '''your code here''', fontdict={'size': 20, 'color':  'red'})
        plt.text(-0.5, -2.3, 'dropout loss=%.4f' % '''your code here''', fontdict={'size': 20, 'color': 'blue'})
        plt.legend(loc='upper left'); plt.ylim((-2.5, 2.5));#plt.pause(0.1)
        plt.show()
        
        # вернемся к режиму train
        net_overfitting.train()
        net_dropped.train()

# Выведим ошибки двух сетей попарно на y:
print('overfitting loss train', 'dropout loss train')
for i in range(len(error)):
    print('%.4f\t\t\t%.4f' % (error[i][0], error[i][1]))

print()
# Выведим ошибки двух сетей попарно на test_y:
print('overfitting loss', '   dropout loss')
for i in range(len(error)):
    print('%.4f\t\t\t%.4f' % '''your code here''')

# Вопросы:
1. Какая суммарная квадратичная ошибка между $y$ и $test\_y$? (c помощью loss_func)
<br>Ответ: ...
2. Сколько параметров у линейного скрытого слоя (не забудьте про свободный член b)?
<br> Ответ: ...
3. Какая у вас получилась ошибка $overfitting\:loss\:train$ в итоге (округлите до 2 знаков после запятой)?
<br> Ответ: ...
4. Какая у вас получилась ошибка $dropout\:loss\:train$ в итоге (округлите до 2 знаков после запятой)?
<br> Ответ: ...
5. Какая у вас получилась ошибка $overfitting\:loss$ в итоге (округлите до 2 знаков после запятой)?
<br> Ответ: ...
6. Какая у вас получилась ошибка $dropout\:loss$ в итоге (округлите до 2 знаков после запятой)?
<br> Ответ: ...

<br> Сравните 3, 4 и подумайте, почему так получилось?
<br> Сравните 4, 5 и подумайте, почему так получилось?
<br> Так же сравните 4, 5 с предыдущими двумя ошибками.